# Code to create the new video with detection

Libraries

In [1]:
import cv2
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

Loading video

In [2]:
# Define the relative path to the video file
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
video_path = project_root / "data" / "recording_2" / "Recording_2_normal_speed.mp4"
video_path = str(video_path)  # Convert Path object to string for OpenCV

# Load the video
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

Defining output

In [ ]:
# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the output video file path
output_video_path = project_root / "data" / "recording_2" / "output_detected_test_1.mp4"
output_video_path = str(output_video_path)

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

Creating output

In [4]:
# Background Subtraction
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)
bg_subtractor.setVarThresholdGen(25)

for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))-40):
    ret, frame_bg = cap.read()
    if not ret: 
        print("Warning: Could not read frame during warm-up.")
        break
    bg_subtractor.apply(frame_bg)

# Initial settings
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
frame_count = 0

# Creating the new frames
while frame_count < cap.get(cv2.CAP_PROP_FRAME_COUNT):
    ret, frame = cap.read()
    if not ret: 
        print("Error: Could not read frame from video.")
        break

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(frame)
    kernel = np.ones((5, 5), np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.medianBlur(fg_mask, 5)
    blurred = cv2.GaussianBlur(fg_mask, (9, 9), 2)

    # Hough Circle Detection
    circles = cv2.HoughCircles(
        blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=100,
        param1=50, param2=30, minRadius=30, maxRadius=100
    )

    # Draw and process only the first detected circle
    output = frame.copy()
    if circles is not None:
        circles = np.uint16(np.around(circles))
        
        # Process only the first circle
        first_circle = circles[0, 0]
        x, y, r = first_circle
        cv2.circle(output, (x, y), r, (0, 255, 0), 3)
        cv2.circle(output, (x, y), 2, (0, 0, 255), 3)
    
    # Write the frame with detected circles to the output video
    out.write(output)
    frame_count += 1

# Check
print(f"Processed {frame_count} frames.")
cap.release() 
out.release()

Processed 276 frames.


If I want to take only the region of the ball:

In [5]:
'''
# Before the while loop
ball_images = []  # List to store extracted ball images

# In the while loop (after the drawing circles)
ball = frame[y-r:y+r, x-r:x+r]
if ball.size > 0:
    ball_rgb = cv2.cvtColor(ball, cv2.COLOR_BGR2RGB)
    ball_images.append(ball_rgb)'
'''

"\n# Before the while loop\nball_images = []  # List to store extracted ball images\n\n# In the while loop (after the drawing circles)\nball = frame[y-r:y+r, x-r:x+r]\nif ball.size > 0:\n    ball_rgb = cv2.cvtColor(ball, cv2.COLOR_BGR2RGB)\n    ball_images.append(ball_rgb)'\n"